# Grover's Algorithm

### Details @ https://en.wikipedia.org/wiki/Grover%27s_algorithm

# Imports

In [ ]:
import qckt
from qckt.backend import *
import numpy as np
import random as rnd

# Qubits assignment for the algorithm

In [ ]:
ufinpsz = 6
inpreg = qckt.QRegister(ufinpsz)
outreg = qckt.QRegister(1)
clmeas = qckt.CRegister(ufinpsz)
nqbits,ncbits,_,_ = qckt.placement(outreg,inpreg,clmeas)

# The 'needle' in the haytack to be searched = marker

In [ ]:
marker = int(rnd.random() * (2**(nqbits-1)-1))
print(("Marker to search = {0:0"+str(nqbits-1)+"b}, ({0:d})").format(marker))

# The oracle circuit

In [ ]:
uf_ckt = qckt.QCkt(nqbits,name="Uf")
uf_ckt.Border()
x_list = []
for i in range(len(inpreg)):
	if (marker & (0b1<<i)) == 0:
		x_list.append(inpreg[-i-1]) # index i backwards from the end of inpreg
if len(x_list) > 0:
	uf_ckt.X(x_list)
uf_ckt.CX(*(inpreg+outreg)) # target of the CX operation is outreg qubit
if len(x_list) > 0:
	uf_ckt.X(x_list)
uf_ckt.Border()
uf_ckt.draw()

# The diffuser circuit

In [ ]:
amp_ckt = qckt.QCkt(nqbits,name="Diffuser")
amp_ckt.H(inpreg)
amp_ckt.X(inpreg)
amp_ckt.CX(*(inpreg+outreg))   ## This is how Umesh Vazirni explains it
# amp_ckt.CZ(*inpreg) ## Gives identical results ... since still invertig phase of the inputs register.
amp_ckt.X(inpreg)
amp_ckt.H(inpreg)
amp_ckt.draw()

# The initializer circuit

In [ ]:
init_ckt = qckt.QCkt(nqbits,name="Initialize")
init_ckt.H(inpreg)
# setup the result qubit in |-> state for phase kickback
init_ckt.X(outreg)
init_ckt.H(outreg)
init_ckt.draw()

# Assemble the full Grover's algorithm circuit

In [ ]:
fullckt = qckt.QCkt(nqbits,ncbits,name="Full Grover's Circuit")
fullckt = fullckt.append(init_ckt)

fullckt.Probe("after initialization", probestates=[marker])
numitrs = int((np.pi/4.0) * (2.0**((nqbits-1.0)/2.0))) # optimal # iter, less or more dont work
print("Number of Invert-Diffuser iterations = ",numitrs)

for itr in range(numitrs):
	fullckt = fullckt.append(uf_ckt)
	fullckt = fullckt.append(amp_ckt)
	fullckt.Probe("after iteration "+str(itr+1), probestates=[marker])
fullckt.M(inpreg,clmeas)
fullckt.draw()

# Run the circuit multiple times, display the stats of readouts

In [ ]:
job = qckt.Job(fullckt, shots=100)
bk = Qeng()  # to run multiple shots, and see stats of readouts
# bk = Qdeb()  # to see Probe on state of interest
bk.runjob(job)
_ = job.plot_counts()